In [23]:
import helper_tools.parser as parser
import importlib

importlib.reload(parser)

relation_df, entity_df, docs = parser.rebel_parser('./datasets/REBEL/en_train.jsonl', 10)

100%|██████████| 10/10 [00:00<00:00, 4223.02it/s]


In [24]:
docs.head()

,docid,text
0,30111982,The Philippine one hundred-peso note (Filipino...
9,30111996,The Philippine five hundred-peso note (Filipin...
20,30111927,"Éamonn Ó Catháin is an Irish chef, sometime au..."
59,30112002,The Philippine one thousand-peso note (Filipin...
70,30112004,Havnar Róðrarfelag is a Faroese rowing club in...


In [15]:
predicate_set_df = relation_df[["predicate", "predicate_uri"]].drop_duplicates()
predicate_set_df

,predicate,predicate_uri
0,face value,P3934
1,position held,P39
5,country,P17
7,has list,P2354
10,country of origin,P495
11,genre,P136
12,location of formation,P740
13,inception,P571
15,work period (start),P2031
16,start time,P580


In [16]:
from langgraph.types import Command
from typing import Literal
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_ollama import OllamaEmbeddings

model = ChatOllama(model="phi4")
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [17]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

documents = []
entity_set = entity_df[['entity', 'entity_uri']].drop_duplicates()
for index, row in entity_set.iterrows():
    documents.append(Document(
        page_content=row["entity"],
        metadata={"uri": row["entity_uri"]},
    ))
    
vector_store.add_documents(documents=documents)

['1beb4394-bc8d-4f3c-b77f-5cbefad0f89f',
 'cbdbe266-1c51-41b0-84cb-cda6862cda09',
 '0e6dcaf8-fbcc-4582-a53f-4af2bbc4de5c',
 '8c2c0c7c-c5db-4ff8-8b61-dd08360a58f1',
 '608153c5-90bd-48f5-a2ac-59ef2f7ba447',
 '8712168f-e5f0-4a6d-b919-066688ae23b2',
 '1a5e1ee3-70c8-4194-bdd8-57766a91db73',
 '8f217c94-3d5b-4873-95df-0c245b1786f1',
 'a39b3bcd-3fde-4416-8cfd-34829d98113a',
 '048a05b3-aeb2-40cc-a960-17d37c0c3444',
 'e1a78708-490c-4e41-9b24-d3c732d65f99',
 'a3be45ae-a0cd-4764-a525-aa28567c44ce',
 '48e01912-8201-4cb1-9459-10ab00d402af',
 'aa6500b3-74b7-45c0-ac5d-79005782e061',
 '41f09390-0697-4571-a011-5110e89a2112',
 '5fd3a056-2274-4876-8fb4-faf32e324a64',
 '0a0c81ca-99a7-4269-9382-133642820012',
 '94faf3ae-1860-4f53-992f-688db8714c60',
 '0b30c788-b1a4-4343-b8b9-a96db0c98f5f',
 'e47af5d6-b146-4f4a-9273-a2fd6ecb879c',
 '814ed930-4c04-4433-9641-97dcd4c2e7f2',
 'fb02a0ac-63cf-4f11-a8f8-5b738c6ab176',
 '8829b1c8-6818-4f7c-a918-377a85ac7e0b',
 '722d9256-8ab9-4c33-a033-7dabba2ab6e9',
 '4df57ccd-d97f-

In [18]:
from langchain_core.prompts import PromptTemplate
import re

prompt_template = PromptTemplate.from_template("""
You are an agent to extract entities out of a given text. The entities will be linked to a knowledge graph later. You should return a list of any explicit and implicit entities found in the text. Please enclose the list with <list> and </list> tags and the entities with <entity> and </entity> tags.

Text: {text}
""")

doc_id = docs.iloc[6]["docid"]
text = docs.iloc[6]["text"]
doc_relation_df = relation_df[relation_df["docid"] == doc_id]
prompt = prompt_template.invoke({"text": text})
print(prompt)
output = model.invoke(prompt)
print(output.content)
entity_list = re.findall(r'<entity>(.*?)</entity>', output.content)
entity_list = [vector_store.similarity_search(x, k=1)[0] for x in entity_list]
entity_set_df = pd.DataFrame({"entity": [x.page_content for x in entity_list], "entity_uri": [x.metadata["uri"] for x in entity_list]})
entity_set_df

text='\nYou are an agent to extract entities out of a given text. The entities will be linked to a knowledge graph later. You should return a list of any explicit and implicit entities found in the text. Please enclose the list with <list> and </list> tags and the entities with <entity> and </entity> tags.\n\nText: The Ministry of Macedonia and Thrace () is a former ministry of Greece. Responsible for the regions of Macedonia and Thrace, since 2015 it has been demoted to the level of a sub-ministry within the Ministry of the Interior. The incumbent Deputy Minister for Macedonia and Thrace is of New Democracy.\n\nThe ministry had been known as the Ministry of Northern Greece until it was renamed on 19 August 1988. It was demoted to a general secretariat in 2009, but was re-established as a ministry in 2012, and again demoted to a sub-ministry within the Ministry of the Interior on 27 January 2015. It is housed in Government House in Thessaloniki.\n'
<list>
  <entity>The Ministry of Mace

,entity,entity_uri
0,Ministry of Macedonia and Thrace,Q3555742
1,Greece,Q41
2,Macedonia,Q81734
3,Ministry of the Interior,Q3555550
4,New Democracy,Q223153
5,Deputy Minister for Macedonia and Thrace,Q12885643
6,Ministry of Macedonia and Thrace,Q3555742
7,19 August 1988,1988-08-19T00:00:00Z^^http://www.w3.org/2001/X...
8,ministry,Q192350
9,2009,+2009^^http://www.w3.org/2001/XMLSchema#decimal


In [19]:
prompt_template = PromptTemplate.from_template(
    """
    
You are a relation extraction agent. Your task is to read a given text (along with a given list of possible entities and relations) and extract the relations found in the text. Each relation should be written in this exact format: <relation>subject;predicate;object</relation>. Please output just the relations using the URIs, nothing else.

Example:

Entity List: 
    entity           entity_uri
0   Olaf Scholz      Q1
1   Germany          Q2

Relation Set: 
    predicate           predicate_uri
0   is chancellor of    P1

Text:
Olaf Scholz is chancellor of Germany.

Expected Output:
<relation>Q1;P1;Q2</relation>

-- Start of input --

Entity Set: 
{entity_set}

Relation Set: 
{relation_set}

Text: 
{text}
    
    """)
prompt = prompt_template.invoke({"text": relation_df["text"].iloc[0], "relation_set": predicate_set_df.to_string(), "entity_set": entity_set_df.to_string()})
print(prompt.text)
output = model.invoke(prompt)
print(output.content)
relation_list = re.findall(r'<relation>(.*?)</relation>', output.content)
relation_list = [relation.split(";") for relation in relation_list]
print(relation_list)


    
You are a relation extraction agent. Your task is to read a given text (along with a given list of possible entities and relations) and extract the relations found in the text. Each relation should be written in this exact format: <relation>subject;predicate;object</relation>. Please output just the relations using the URIs, nothing else.

Example:

Entity List: 
    entity           entity_uri
0   Olaf Scholz      Q1
1   Germany          Q2

Relation Set: 
    predicate           predicate_uri
0   is chancellor of    P1

Text:
Olaf Scholz is chancellor of Germany.

Expected Output:
<relation>Q1;P1;Q2</relation>

-- Start of input --

Entity Set: 
                                      entity                                                       entity_uri
0           Ministry of Macedonia and Thrace                                                         Q3555742
1                                     Greece                                                              Q41
2       

In [20]:
pred_relation_df = pd.DataFrame(relation_list, columns=["subject_uri", "predicate_uri", "object_uri"])
pred_relation_df

,subject_uri,predicate_uri,object_uri
0,Q3555742,P39,Q12885643


In [21]:
correct_relation_df = pred_relation_df.merge(doc_relation_df[["subject_uri", "predicate_uri", "object_uri"]], on=["subject_uri", "predicate_uri", "object_uri"], how="inner")
correct_relation_df

,subject_uri,predicate_uri,object_uri


In [22]:
print(f"Accuracy: {len(correct_relation_df) / len(pred_relation_df)}")
print(f"Precision: {len(correct_relation_df) / len(doc_relation_df)}")

Accuracy: 0.0


ZeroDivisionError: division by zero

In [35]:
relation_df

,docid,text,subject,subject_uri,predicate,predicate_uri,object,object_uri
0,30111982,The Philippine one hundred-peso note (Filipino...,Philippine one hundred-peso note,Q7185360,face value,P3934,100,+100^^http://www.w3.org/2001/XMLSchema#decimal
1,30111982,The Philippine one hundred-peso note (Filipino...,Manuel A. Roxas,Q223419,position held,P39,Philippine president,Q1209571
2,30111996,The Philippine five hundred-peso note (Filipin...,Philippine five hundred-peso note,Q7185306,face value,P3934,500,+500^^http://www.w3.org/2001/XMLSchema#decimal
3,30111996,The Philippine five hundred-peso note (Filipin...,Corazon Aquino,Q1480,position held,P39,President,Q1209571
4,30112002,The Philippine one thousand-peso note (Filipin...,Philippine one thousand-peso note,Q7185363,face value,P3934,1000,+1000^^http://www.w3.org/2001/XMLSchema#decimal
5,30112004,Havnar Róðrarfelag is a Faroese rowing club in...,Tórshavn,Q10704,country,P17,Faroese,Q4628
6,30111989,The Philippine two hundred-peso note (Filipino...,Philippine two hundred-peso note,Q7185399,face value,P3934,200,+200^^http://www.w3.org/2001/XMLSchema#decimal
7,30111989,The Philippine two hundred-peso note (Filipino...,Philippine president,Q1209571,has list,P2354,14th President of the Philippines,Q269860
8,30111989,The Philippine two hundred-peso note (Filipino...,Diosdado Macapagal,Q312539,position held,P39,Philippine president,Q1209571
9,30111989,The Philippine two hundred-peso note (Filipino...,President Gloria Macapagal-Arroyo,Q123665,position held,P39,Philippine president,Q1209571
